In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

X = torch.randn(2, 20)
net(X)

tensor([[ 0.0635,  0.5039, -0.0070,  0.2556, -0.3379, -0.4444, -0.3287, -0.2093,
          0.0252, -0.3500],
        [ 0.3568,  0.2414,  0.0174, -0.0027,  0.0950, -0.2753, -0.0480, -0.2939,
          0.4303, -0.0855]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [4]:
net = MLP()
net(X)

tensor([[ 0.1761, -0.1158, -0.5057,  0.2418,  0.2494,  0.1285,  0.1211, -0.2847,
          0.0753, -0.5586],
        [-0.2284, -0.0816, -0.1280,  0.1040, -0.0059,  0.0749,  0.0436, -0.2471,
         -0.1817, -0.1451]], grad_fn=<AddmmBackward0>)

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

net(X)

tensor([[-0.2396,  0.4969, -0.1922, -0.0052,  0.0116,  0.1915, -0.2237,  0.2193,
          0.3994, -0.3578],
        [-0.2149,  0.6349,  0.1267,  0.3126, -0.1521, -0.3008, -0.3116,  0.1082,
          0.1257, -0.0493]], grad_fn=<AddmmBackward0>)

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=True)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [9]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1148, grad_fn=<SumBackward0>)

In [10]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    

In [11]:
chimera = nn.Sequential(
    NestMLP(),
    nn.Linear(16, 20),
    FixedHiddenMLP()
)
chimera(X)

tensor(0.4568, grad_fn=<SumBackward0>)